## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Labuhan,ID,-6.8844,112.2051,81.46,70,100,3.85,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,53.13,86,99,1.59,overcast clouds
2,2,East London,ZA,-33.0153,27.9116,58.28,73,70,4.85,broken clouds
3,3,Tezu,IN,27.9167,96.1667,78.55,90,61,0.89,broken clouds
4,4,Matara,LK,5.9485,80.5353,77.52,86,21,9.15,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Tezu,IN,27.9167,96.1667,78.55,90,61,0.89,broken clouds
4,4,Matara,LK,5.9485,80.5353,77.52,86,21,9.15,few clouds
6,6,Inirida,CO,3.8653,-67.9239,78.19,82,73,2.46,broken clouds
10,10,Pitimbu,BR,-7.4706,-34.8086,73.99,80,5,7.07,clear sky
11,11,Puerto Ayora,EC,-0.7393,-90.3518,66.54,93,77,8.01,broken clouds
15,15,Natal,BR,-5.7950,-35.2094,77.22,83,0,6.91,clear sky
17,17,Hobart,AU,-42.8794,147.3294,60.06,66,0,8.05,clear sky
18,18,Rikitea,PF,-23.1203,-134.9692,73.04,76,0,15.17,clear sky
21,21,Albany,US,42.6001,-73.9662,64.17,84,74,3.00,broken clouds
23,23,Avarua,CK,-21.2078,-159.7750,69.85,94,100,17.27,light rain


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                314
City                   314
Country                313
Lat                    314
Lng                    314
Max Temp               314
Humidity               314
Cloudiness             314
Wind Speed             314
Current Description    314
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Tezu,IN,27.9167,96.1667,78.55,90,61,0.89,broken clouds
4,4,Matara,LK,5.9485,80.5353,77.52,86,21,9.15,few clouds
6,6,Inirida,CO,3.8653,-67.9239,78.19,82,73,2.46,broken clouds
10,10,Pitimbu,BR,-7.4706,-34.8086,73.99,80,5,7.07,clear sky
11,11,Puerto Ayora,EC,-0.7393,-90.3518,66.54,93,77,8.01,broken clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tezu,IN,78.55,broken clouds,27.9167,96.1667,
4,Matara,LK,77.52,few clouds,5.9485,80.5353,
6,Inirida,CO,78.19,broken clouds,3.8653,-67.9239,
10,Pitimbu,BR,73.99,clear sky,-7.4706,-34.8086,
11,Puerto Ayora,EC,66.54,broken clouds,-0.7393,-90.3518,
15,Natal,BR,77.22,clear sky,-5.7950,-35.2094,
17,Hobart,AU,60.06,clear sky,-42.8794,147.3294,
18,Rikitea,PF,73.04,clear sky,-23.1203,-134.9692,
21,Albany,US,64.17,broken clouds,42.6001,-73.9662,
23,Avarua,CK,69.85,light rain,-21.2078,-159.7750,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df = hotel_df.dropna()
hotel_df['Hotel Name'].value_counts()

Circuit House                                                1
Anemon Denizli Hotel                                         1
عمارت ساعتی تفت                                              1
Hotel Alex Johnson Rapid City, Curio Collection by Hilton    1
املاك خالد الشعشوع                                           1
                                                            ..
Holiday Inn Express Lompoc, an IHG Hotel                     1
El Medina                                                    1
Hampton Inn Cedar City                                       1
Shanchengxuan Hotel                                          1
Palais du chef Canton Salamt                                 1
Name: Hotel Name, Length: 286, dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
hotel_df = pd.read_csv(output_data_file)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather/dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))